In [1]:
import nilearn
from nilearn import image
import numpy as np
import pandas as pd
import math
import gc

In [2]:
#Inputs
input_file = "/Volumes/GoogleDrive/My Drive/UW /Brain Path map/BVIS/VOI atlas/WMT_HCP_YA1065_tractography/nifti/association/C_FP_L.nii.gz"
#filename_csv = "MNI_152_2009a_wm.csv"

filename_txt = "WMT_MNI_2009b_C_FP_L.txt"

key1 = "WMT_C_FP_L_int"

In [4]:
name = input_file.split('.nii.gz')[0]
print(name)
filename_txt = f'WMT_MNI_2009b_{name}.txt'
print(filename_txt)

/Volumes/GoogleDrive/My Drive/UW /Brain Path map/BVIS/VOI atlas/WMT_HCP_YA1065_tractography/nifti/association/C_FP_L
WMT_MNI_2009b_/Volumes/GoogleDrive/My Drive/UW /Brain Path map/BVIS/VOI atlas/WMT_HCP_YA1065_tractography/nifti/association/C_FP_L.txt


In [25]:
#Loading MNI file
a = nilearn.image.load_img(input_file, wildcards=True, dtype=None)
data_obj = a.get_data()

/var/folders/5x/tckh2qyj2vb_8rv_m0ky_wb00000gp/T/ipykernel_66103/2979440676.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data_obj = a.get_data()


In [26]:
##Validating shape of MNI image
print(data_obj.shape)
check_shape = data_obj.shape
valid_dims = [197,233,189]
n_for_check = len(valid_dims)
for k, l in zip(check_shape[:n_for_check], valid_dims):
    print(k==l)


(314, 378, 272)
False
False
False


In [27]:

n_i, n_j, n_k = data_obj.shape
center_i = (n_i - 1) // 2  # // for integer division
center_j = (n_j - 1) // 2
center_k = (n_k - 1) // 2
center_i, center_j, center_k
center_vox_value = data_obj[center_i, center_j, center_k]
print(center_vox_value, center_i, center_j, center_k)
type(center_i)
print(data_obj[100,200,100])

0 156 188 135
0


In [28]:
#Extracting coordinates and their values
rows = n_i*n_j*n_k
vox_extract = np.empty([rows, 4])
index = 0;
for i in range(n_i):
    for j in range(n_j):
        for k in range(n_k):
            vox_extract[index][0] = math.floor(i)
            vox_extract[index][1] = math.floor(j)
            vox_extract[index][2] = math.floor(k)
            vox_extract[index][3] = data_obj[i,j,k]
            index = index+1;
df = pd.DataFrame(vox_extract)


In [29]:
#Check df before saving to file
df = df.astype({0:int,1:int,2:int})

In [30]:
df = df.rename(columns={0:"X",1:"Y",2:"Z",3:key1})
print(df)

print("----min----")
print(df[["X","Y","Z",key1]].min())
print("----max----")
print(df[["X","Y","Z",key1]].max())

count = (df[key1] == 0).sum()
print ('Vox with zero value:', count)

            X    Y    Z  WMT_C_FP_L_int
0           0    0    0             0.0
1           0    0    1             0.0
2           0    0    2             0.0
3           0    0    3             0.0
4           0    0    4             0.0
...       ...  ...  ...             ...
32284219  313  377  267             0.0
32284220  313  377  268             0.0
32284221  313  377  269             0.0
32284222  313  377  270             0.0
32284223  313  377  271             0.0

[32284224 rows x 4 columns]
----min----
X                 0.0
Y                 0.0
Z                 0.0
WMT_C_FP_L_int    0.0
dtype: float64
----max----
X                 313.0
Y                 377.0
Z                 271.0
WMT_C_FP_L_int      1.0
dtype: float64
Vox with zero value: 32199012


In [31]:
#Saving data to a text file
df.to_csv(filename_txt)

In [32]:
del df
del vox_extract
del data_obj
gc.collect()

15